## Preprocessing the Data

In [2]:
import pandas as pd
import numpy as np

### Step 1: Filter Breast Cancer Cell Lines (Cell_Lines_Details.csv)

#### Goal is to get only the breast cancer lines
- We need to get the COSMIC ID from one file and match it from the ones in the other file that have the site as 'breast'

In [3]:
df_details = pd.read_excel('../data/raw_data/Cell_Lines_Details.xlsx')
## COSMIC ID sheet, if needed includes site and histology
## df_COSMIC = pd.read_excel('../data/raw_data/Cell_Lines_Details.xlsx', sheet_name=1)

C:\Users\Arnav Harsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df_details.head(5)

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [5]:
filtered_details_df = df_details[df_details['GDSC\nTissue\ndescriptor 2'] == 'breast']
filtered_details_df.head(5)
print(len(filtered_details_df))

52


N is only 52, therefore we only have 52 samples.
Since we are focusing on just breast cancer this helps us be more consistant and let our results be more interpretable.

### Step 2: Filter rows of GDSC2_FITTED_DOSE_RESPONSE.csv to only contain those cell lines with matching COSMIC ID from filtered_details_df (DRUG RESPONSE)

#### Goal is to match COSMIC IDs with those we got previously with those found in the new file.

In [6]:
GDSC2_df = pd.read_excel('../data/raw_data/GDSC2_fitted_dose_response_27Oct23.xlsx')

In [7]:
filtered_GDSC2_df = GDSC2_df[GDSC2_df['COSMIC_ID'].isin(filtered_details_df['COSMIC identifier'])]

In [8]:
filtered_GDSC2_df.head(5)

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
92,GDSC2,343,15969724,749709,HCC1954,SIDM00872,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,0.317741,0.983262,0.082831,1.403164
93,GDSC2,343,15970009,749710,HCC1143,SIDM00866,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,0.636184,0.969050,0.077198,1.576546
94,GDSC2,343,15970293,749711,HCC1187,SIDM00885,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,1.235544,0.992326,0.079605,1.902878
95,GDSC2,343,15970577,749712,HCC1395,SIDM00884,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,-2.255899,0.861208,0.096794,0.001898
96,GDSC2,343,15970753,749713,HCC1599,SIDM00877,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,-3.247021,0.768404,0.111973,-0.537737


In [9]:
print("Unique cell lines:", filtered_GDSC2_df['COSMIC_ID'].nunique())
print("Total rows:", len(filtered_GDSC2_df))
print("Drugs per cell line (mean):", len(filtered_GDSC2_df) / filtered_GDSC2_df['COSMIC_ID'].nunique())

Unique cell lines: 52
Total rows: 13388
Drugs per cell line (mean): 257.46153846153845


In [10]:
print(filtered_GDSC2_df['COSMIC_ID'].value_counts())

COSMIC_ID
908122     284
749717     284
909907     284
905960     284
910852     284
910948     284
906851     284
925338     284
946382     284
1240172    284
907048     284
946359     283
910927     283
1290905    283
1290922    283
1298157    283
908123     283
908121     283
749709     283
905951     283
749714     283
905945     283
906826     283
905946     283
905957     283
906801     282
749711     282
924240     282
910704     282
906862     282
907047     282
908151     282
906812     282
749710     282
1290798    281
906844     280
909256     280
907045     278
1303900    273
749715     266
907046     257
1330941    256
749713     186
909778     186
1303911    180
749716     178
749712     176
908120     176
924106     173
949093     163
910910     161
1290906     14
Name: count, dtype: int64


This shows that there are x many drugs tested on the cell

So, we have 52 Breast CancerCell Lines but there were many drugs tested for each

### Step 3: Filter columns of cell_line_RMA_proc_basalExp to only keep expression values for COSMIC IDs we have. (EXPRESSION MATRIX)

#### Goal is to match the COSMIC IDs again
- We also have to make sure gene names are rows and the cell lines are columnms.

In [11]:
cell_line_df = pd.read_csv('../data/raw_data/Cell_line_RMA_proc_basalExp.txt', sep='\t')

In [12]:
cell_line_df.head(5)

,GENE_SYMBOLS,GENE_title,DATA.906826,DATA.687983,DATA.910927,DATA.1240138,DATA.1240139,DATA.906792,DATA.910688,DATA.1240135,...,DATA.753584,DATA.907044,DATA.998184,DATA.908145,DATA.1659787,DATA.1298157,DATA.1480372,DATA.1298533,DATA.930299,DATA.905954.1
0,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:11858],7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
1,TNMD,tenomodulin [Source:HGNC Symbol;Acc:17757],2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
2,DPM1,dolichyl-phosphate mannosyltransferase polypep...,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
3,SCYL3,SCY1-like 3 (S. cerevisiae) [Source:HGNC Symbo...,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
4,C1orf112,chromosome 1 open reading frame 112 [Source:HG...,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795


The expression matrix (what we are preprocessing) ...
- Each column is a cancer cell line (Data.<COSMIC ID>)
- We can then use this to match up with the COSMIC IDs we are tracking.
- Each row is a gene
- The values are the gene expression levels
- We will use this dataset as the matrix of features (X)

However, the drug response data GDSC2 ... (Step 2)
- Each row is a measure of drug response for a cell line
- Key column : COSMIC ID is used to help identify cell lines (what we organized by)
- Key Value : LN_IC50, this is the target we are getting from the regression model
- This gives labels for each training sample (y)
- Therefore this is the dependent variable matrix (y)

so the first step would be getting the COSMIC IDs from the gene expression then to filter the file to make sure it dont have duplicates in COSMIC IDs then we sort both files so their values are aligned

In [13]:
COSMIC_list = []

for i in range(2, cell_line_df.shape[1]):
    id = cell_line_df.columns[i].split('.')[1]
    if id in COSMIC_list:
        break
    COSMIC_list.append(id)


print(len(COSMIC_list))
##filtered_cell_line_df 

843


Rename columns and keep only the relevant columns

In [14]:
cols_to_keep = ['GENE_SYMBOLS'] + [f'DATA.{id}' for id in COSMIC_list]
cell_line_df = cell_line_df[cols_to_keep]

cell_line_df.columns = ['GENE_SYMBOLS'] + COSMIC_list

In [15]:
cell_line_df.head(5)

,GENE_SYMBOLS,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,...,946358,1503365,1331028,910910,1660035,909715,1295740,907281,906864,909974
0,TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,...,3.384334,8.631575,8.318344,7.086304,8.077116,3.471177,3.455584,3.190195,7.421901,4.575260
1,TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,...,2.922821,2.691465,2.703581,2.952920,2.781325,2.687947,2.794158,2.933651,2.846323,2.802682
2,DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,...,10.682159,10.403906,11.256023,11.536803,10.038055,10.505151,9.860766,9.829097,10.061235,10.004076
3,SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,...,4.256928,4.184282,3.977725,4.520248,5.205411,4.915178,4.465537,4.267119,3.690842,3.853340
4,C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,...,3.370588,3.638513,3.423418,3.324127,3.758069,4.288071,4.894033,4.728701,3.519667,3.278125


Gene names as row index abd transpose the matrix

In [16]:
cell_line_df.set_index('GENE_SYMBOLS', inplace=True)

cell_line_df = cell_line_df.transpose()
cell_line_df.index.name = 'COSMIC_ID'

In [17]:
cell_line_df.head(5)

GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00526,PPY2,NaN,NaN,KRT18P55,NaN,POLRMTP1,UBL5P2,TBC1D3P5,NaN
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
906826,7.632023,2.964585,10.379553,3.614794,3.380681,3.324692,3.566350,8.204530,5.235118,5.369039,...,6.786925,2.997054,3.109774,7.882377,3.331134,2.852537,3.130696,9.986616,3.073724,7.284733
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,5.317911,3.263745,3.059424,8.681302,2.992611,2.776771,3.260982,9.002814,3.000182,8.504804
910927,8.712338,2.643508,9.880733,3.956230,3.236620,3.241246,2.931034,8.191246,5.426841,5.120747,...,3.143006,3.112145,2.930254,8.707886,2.886574,2.685307,3.176239,9.113243,2.916274,7.059092
1240138,7.797142,2.817923,9.883471,4.063701,3.558414,3.101247,7.211707,8.630643,5.617714,4.996434,...,3.153896,3.151576,2.850726,7.872535,3.812119,3.436412,3.074432,9.958284,3.256500,7.318125
1240139,7.729268,2.957739,10.418840,4.341500,3.840373,3.001802,3.375422,8.296950,5.669418,4.180205,...,3.652660,2.918475,2.849537,8.945953,3.412586,2.951270,3.213545,9.938978,3.396126,7.726867


In [18]:
cell_line_df = cell_line_df.dropna(axis=1,  how='all')

In [19]:
cell_line_df.head(5)

GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00526,PPY2,NaN,NaN,KRT18P55,NaN,POLRMTP1,UBL5P2,TBC1D3P5,NaN
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
906826,7.632023,2.964585,10.379553,3.614794,3.380681,3.324692,3.566350,8.204530,5.235118,5.369039,...,6.786925,2.997054,3.109774,7.882377,3.331134,2.852537,3.130696,9.986616,3.073724,7.284733
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,5.317911,3.263745,3.059424,8.681302,2.992611,2.776771,3.260982,9.002814,3.000182,8.504804
910927,8.712338,2.643508,9.880733,3.956230,3.236620,3.241246,2.931034,8.191246,5.426841,5.120747,...,3.143006,3.112145,2.930254,8.707886,2.886574,2.685307,3.176239,9.113243,2.916274,7.059092
1240138,7.797142,2.817923,9.883471,4.063701,3.558414,3.101247,7.211707,8.630643,5.617714,4.996434,...,3.153896,3.151576,2.850726,7.872535,3.812119,3.436412,3.074432,9.958284,3.256500,7.318125
1240139,7.729268,2.957739,10.418840,4.341500,3.840373,3.001802,3.375422,8.296950,5.669418,4.180205,...,3.652660,2.918475,2.849537,8.945953,3.412586,2.951270,3.213545,9.938978,3.396126,7.726867


In [21]:
cell_line_df.head(6)

GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00526,PPY2,NaN,NaN,KRT18P55,NaN,POLRMTP1,UBL5P2,TBC1D3P5,NaN
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
906826,7.632023,2.964585,10.379553,3.614794,3.380681,3.324692,3.566350,8.204530,5.235118,5.369039,...,6.786925,2.997054,3.109774,7.882377,3.331134,2.852537,3.130696,9.986616,3.073724,7.284733
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,5.317911,3.263745,3.059424,8.681302,2.992611,2.776771,3.260982,9.002814,3.000182,8.504804
910927,8.712338,2.643508,9.880733,3.956230,3.236620,3.241246,2.931034,8.191246,5.426841,5.120747,...,3.143006,3.112145,2.930254,8.707886,2.886574,2.685307,3.176239,9.113243,2.916274,7.059092
1240138,7.797142,2.817923,9.883471,4.063701,3.558414,3.101247,7.211707,8.630643,5.617714,4.996434,...,3.153896,3.151576,2.850726,7.872535,3.812119,3.436412,3.074432,9.958284,3.256500,7.318125
1240139,7.729268,2.957739,10.418840,4.341500,3.840373,3.001802,3.375422,8.296950,5.669418,4.180205,...,3.652660,2.918475,2.849537,8.945953,3.412586,2.951270,3.213545,9.938978,3.396126,7.726867
906792,7.074533,2.889677,9.773987,4.270903,3.815055,3.298915,4.336319,8.838671,5.656988,5.479766,...,5.574324,3.141893,2.987299,8.114437,2.999640,3.233383,3.382112,8.714820,3.497439,7.085595


In [22]:
print(cell_line_df.shape)           # (52, ~17400) — depends on how many genes
print(cell_line_df.index[:5])       # COSMIC_IDs
print(cell_line_df.columns[:5])     # TSPAN6, TNMD, ...

(843, 17737)
Index(['906826', '687983', '910927', '1240138', '1240139'], dtype='object', name='COSMIC_ID')
Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112'], dtype='object', name='GENE_SYMBOLS')


843 cell lines and 17737 genes

Now we need to make sure we have 52 cell lines which are all breast cancer lines, we can use our first dataset to accomplish this

In [23]:
filtered_details_df.head(5)

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
270,AU565,910704.0,Y,Y,Y,Y,Y,breast,breast,BRCA,MSS/MSI-L,R,Adherent
271,BT-20,906801.0,Y,Y,Y,Y,Y,breast,breast,BRCA,MSS/MSI-L,D/F12,Adherent
272,BT-474,946359.0,Y,Y,Y,Y,Y,breast,breast,BRCA,MSS/MSI-L,R,Adherent
273,BT-483,949093.0,Y,Y,Y,Y,Y,breast,breast,BRCA,MSS/MSI-L,R,Adherent
274,BT-549,905951.0,Y,Y,Y,Y,Y,breast,breast,BRCA,MSS/MSI-L,R,Adherent


In [24]:
print(len(filtered_details_df))

52


In [30]:
BRCA_COSMIC_ID = []

for i in range(filtered_details_df.shape[0]):
    id = filtered_details_df.iloc[i]['COSMIC identifier']
    BRCA_COSMIC_ID.append(id)

In [26]:
print(len(BRCA_COSMIC_ID))

52


df = df[df['column_name'].isin(your_list)]

^helpful tool to keep only items found in the list we created

In [ ]:
filtered_cell_line_df = cell_line_df.loc[cell_line_df.index.isin(BRCA_COSMIC_ID)]

In [31]:
filtered_cell_line_df.shape

(0, 17737)

make sure to convert the types maybe thats the issue??

In [32]:
print(type(next(iter(cell_line_df.index))))
print(type(next(iter(BRCA_COSMIC_ID))))

<class 'str'>
<class 'numpy.float64'>


In [33]:
BRCA_COSMIC_ID = [str(int(i)) for i in BRCA_COSMIC_ID]

In [37]:
print(len(BRCA_COSMIC_ID))

52


In [35]:
filtered_cell_line_df = cell_line_df.loc[cell_line_df.index.isin(BRCA_COSMIC_ID)]

In [36]:
filtered_cell_line_df.shape

(44, 17737)

In [38]:
missing_ids = [i for i in BRCA_COSMIC_ID if i not in cell_line_df.index]
print("Missing IDs:", missing_ids)
print("Count:", len(missing_ids))

Missing IDs: ['924106', '749712', '749713', '749717', '1240172', '1298157', '908120', '1303911']
Count: 8


These 8 COSMIC IDs are not found in the expression dataset and their expression data is unavaiable